# Bundled Irreducible Derivatives (BID) Approach


Here we demonstrate computing phonon of rocksalt with a FTG of 2$\hat{1}$, using both BID and HS-BID approaches.

In BID approach, phonons and interactions are computed in the supercell of the FTG.
The chainrule derivatives analysis is used to find the minimum amount of measurements needed to solve for the irreducible derivatives.

In [1]:
import numpy as np

from principia_materia.io_interface.qpoints_io import tuple_to_str
from principia_materia.io_interface.vasp import parse_poscar
from principia_materia.io_interface.jobs_db import JobsDB
from principia_materia.translation_group import (
    get_structure,
    CrystalFTG,
    save_structure,
    )
from principia_materia.phonon_id.bid import BID
from principia_materia.phonon_id.hsbid import HSBID

In [2]:
POSCAR = "./nacl_hsbid/POSCAR"

ORDER = 2
PG = "Oh"
MESH = np.diag([2, 2, 2])
DELTA = np.arange(0.02, 0.21, 0.02)

COMPUTE_ENGINE = "vasp"

ROOT_DIR = "."

In [3]:
structure = get_structure(parse_poscar(POSCAR), stype=CrystalFTG)
structure.orbitals = "p"

In [4]:
job_handler= JobsDB(
  structure=structure,
  root_directory=ROOT_DIR,
  db_path="nacl_hsbid/database.db",
  db_type="sqlite",
  )

job_handler.set_table("bid_phonon")

In [5]:
bid = BID(
  structure=structure,
  supa=MESH,
  order=ORDER,
  pg=PG,
  )
bid.set_chainrule_derivatives(verbose=True)
bid._set_displacement_generator()
bid.find_measurements(verbose=True)

Computing ADT for Qpoint: ((0, 0, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 0, 0), (1/2, 0, 0))
Computing ADT for Qpoint: ((1/2, 1/2, 0), (1/2, 1/2, 0))
Summarize irreducible derivatives.
Compute vectorized tensors.
Number of unknowns:  11
Rank of single measurements:  11
Estimated number of measurements:  1


In [6]:
bid.create_jobs(
    job_handler=job_handler,
    deltas=DELTA,
    append=True,
    dry_run=True,
    )

# Hierarchical Supercell BID Approach

In HS-BID approach, the minimum amount of subcells of the FTG are found to satisfy that every irreducible derivative can be computed in the smallest possible supercell.
Then for these subcells, chainrule derivatives analysis is used to find the minimum measurements to solve for irreducible derivatives.

In [7]:
hsbid = HSBID(
  structure=structure,
  supa=MESH,
  order=ORDER,
  pg=PG,
  )
hsbid.set_chainrule_derivatives(verbose=True)
hsbid.set_hsbid()
hsbid.find_measurements(verbose=True)

Computing ADT for Qpoint: ((0, 0, 0), (0, 0, 0))
Computing ADT for Qpoint: ((1/2, 0, 0), (1/2, 0, 0))
Computing ADT for Qpoint: ((1/2, 1/2, 0), (1/2, 1/2, 0))
Summarize irreducible derivatives.
Compute vectorized tensors.
Find measurements for supercell # 0
Number of unknowns:  1
Rank of single measurements:  1
Estimated number of measurements:  1
Find measurements for supercell # 1
Number of unknowns:  4
Rank of single measurements:  4
Estimated number of measurements:  1
Find measurements for supercell # 2
Number of unknowns:  6
Rank of single measurements:  5
Estimated number of measurements:  2


In [8]:
hsbid.create_jobs(
    job_handler=job_handler,
    deltas=DELTA,
    append=True,
    dry_run=True,
    )

In [9]:
hsbid.set_results(
    job_handler=job_handler,
    data_type="forces",
    )

In [10]:
hsbid.set_errortail_results(
    pick_min=3,
    pick_max=5,
    seperate_complex=True,
    #  output="out_errortail.h5",
    )

In [11]:
hsbid.compute_irreducible_derivatives()

In [12]:
dynamic_tensors = hsbid.get_dynamic_tensors()

In [13]:
for i in range(len(hsbid._chainrule.QpointsN.irreducible_Qpoints)):
    print(tuple_to_str(hsbid._chainrule.QpointsN.irreducible_Qpoints[i]))
    print(dynamic_tensors[hsbid._chainrule.QpointsN.irreducible_Qpoints_Qind].real[i])
    print()

((0, 0, 0), (0, 0, 0))
[[ 1.2163185  0.         0.        -1.2163185  0.         0.       ]
 [ 0.         1.2163185  0.         0.        -1.2163185  0.       ]
 [ 0.         0.         1.2163185  0.         0.        -1.2163185]
 [-1.2163185  0.         0.         1.2163185  0.         0.       ]
 [ 0.        -1.2163185  0.         0.         1.2163185  0.       ]
 [ 0.         0.        -1.2163185  0.         0.         1.2163185]]

((1/2, 0, 0), (1/2, 0, 0))
[[ 2.18264725 -0.80163822 -0.80163822  0.          0.          0.        ]
 [-0.80163822  2.18264725  0.80163822  0.          0.          0.        ]
 [-0.80163822  0.80163822  2.18264725  0.          0.          0.        ]
 [ 0.          0.          0.          2.30057778 -0.76845191 -0.76845191]
 [ 0.          0.          0.         -0.76845191  2.30057778  0.76845191]
 [ 0.          0.          0.         -0.76845191  0.76845191  2.30057778]]

((1/2, 1/2, 0), (1/2, 1/2, 0))
[[ 1.47854869  0.          0.          1.0212613   

## Comparison between HS-BID and LID of Phonon and DOS of NaCl with FTG 4$\hat{1}$ including LO-TO splitting

![Phonon and DOS of NaCl](nacl_hsbid/figure.pdf)